In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from pathlib import Path
import os
import sys
import bz2
import pickle
np.random.seed(10)
# adds the visibility of the mlem module, needed to load the attack models
sys.path.append("../../../") 
import mlem

In [2]:
import warnings

In [3]:
EXPERIMENTS_FOLDER = "/home/gerardozinno/Desktop/risultati_clustered_lhs/lhs/same"

In [4]:
def get_attack_mod(index:int, targets=[0,1], base_folder=EXPERIMENTS_FOLDER):
    """
    Args:
        index - row of the dataset on which the attack model was built
        targets (int | List(int)) - targets for the attack model.
    Returns:
        List containing the attack models (or model) for that particular index
    """
    if type(targets) is int:
        targets = [targets]
    loaded_models = []
    for t in targets:
        path = f"{EXPERIMENTS_FOLDER}/{index}/attack/{t}/model.pkl.bz2"
        with bz2.BZ2File(path) as f:
            data = pickle.load(f)
            loaded_models.append(data)
    return loaded_models
        
def get_local_model(index:int, base_folder=EXPERIMENTS_FOLDER):
    """
    Args:
        index - row of the dataset on which the local model was built.
    Returns:
        Local model
    """
    path = f"{EXPERIMENTS_FOLDER}/{index}/black_box"
    local_model_path = f"{path}/model.pkl.bz2"
    with bz2.BZ2File(local_model_path) as lm:
        local_model = pickle.load(lm)
    return local_model

def get_local_model_data(index: int, base_folder=EXPERIMENTS_FOLDER):
    """
    Args:
        index - row of the dataset on which the local model was built.
    Returns:
        Structure with keys x and y
    """
    path = f"{EXPERIMENTS_FOLDER}/{index}/black_box"
    data_path = f"{path}/data.npz"
    loaded = np.load(data_path, allow_pickle=True)
    return loaded

def get_subset_target_cluster(df, target, cluster):
    """
    Returns the subset of a dataframe with the specified target and cluster.
    """
    tmp = df[df['Target'] == target]
    return tmp[tmp['Cluster'] == cluster]

def split_probs_array(arr):
    """
    
    """
    n_classes = len(arr[0])
    separated = []
    max_index_row = np.argmax(arr, axis=1)
    for c in range(n_classes):
        separated.append(arr[max_index_row == c])
    return separated

loading the data and the black box

In [5]:
with bz2.BZ2File("./dataset_backups/adult_randfor.bz2") as f:
    black_box = pickle.load(f)

/home/gerardozinno/.pyenv/versions/3.9.9/envs/ml-environment/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/gerardozinno/.pyenv/versions/3.9.9/envs/ml-environment/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.0.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [6]:
loaded = np.load("adult_randfor.data-extended.npz", allow_pickle=True)
for k in loaded.keys():
    print(k, end=" ")

x_train x_test y_train y_test x_test_clustered y_test_clustered 

In [7]:
x_train = loaded["x_train"]
x_test = loaded["x_test"]
y_train = loaded["y_train"]
y_test = loaded["y_test"]
x_test_clustered = loaded["x_test_clustered"]
y_test_clustered = loaded["y_test_clustered"]

doing some integrity checks

In [8]:
feat_cols = [
    'Age', 'Workclass', 'Fnlwgt', 'Education', 'Education-num', 'Marital-status',
    'Occupation', 'Relationship', 'Race', 'Sex', 'Capital-gain', 'Capital-loss',
    'Hours-per-week', 'Native-country'
]
adult_train = pd.read_csv("adult_trainset.csv")
adult_test = pd.read_csv("adult_testset.csv")
adult_test_subset_cluster = pd.read_csv("adult_clustered_5_closest.csv")
assert (adult_train[feat_cols] == x_train).all().all() and \
       (adult_test[feat_cols] == x_test).all().all()   and \
       (adult_test_subset_cluster[feat_cols] == x_test_clustered).all().all()

In [9]:
adult_train_ones = adult_train[adult_train['Target'] == 1]
adult_train_zeroes = adult_train[adult_train['Target'] == 0]

In [10]:
adult_train_ones

,Unnamed: 0,Age,Workclass,Fnlwgt,Education,Education-num,Marital-status,Occupation,Relationship,Race,Sex,Capital-gain,Capital-loss,Hours-per-week,Native-country,Target,Cluster
12674,26860,1.032523,0.218918,-0.310837,0.164328,-0.440434,0.455011,0.061527,0.455728,0.263855,0.313955,-0.147502,-0.218673,-0.078031,0.254411,1,1
12675,26865,0.880215,0.218918,1.421195,0.263198,0.344281,0.455011,0.225590,0.455728,0.263855,0.313955,-0.147502,-0.218673,-0.078031,0.254411,1,1
12676,4411,0.347133,0.218918,1.082561,0.564576,1.521353,0.455011,0.448686,0.455728,0.263855,0.313955,-0.147502,-0.218673,0.756794,0.254411,1,3
12677,24148,0.575597,0.218918,-0.042629,0.056298,-1.225149,0.455011,0.061527,0.455728,0.263855,0.313955,-0.147502,-0.218673,0.339381,0.254411,1,1
12678,28103,1.032523,0.285829,-0.062930,0.076923,-0.832791,0.455011,0.485342,0.455728,0.263855,0.313955,-0.147502,-0.218673,0.756794,0.254411,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16871,6010,-0.262102,0.218918,0.001145,0.421658,1.128996,0.455011,0.270647,0.493599,0.263855,0.113783,-0.147502,-0.218673,-0.912857,0.254411,1,3
16872,24384,0.956369,0.218918,0.388645,0.200180,-0.048076,0.455011,0.225590,0.455728,0.263855,0.313955,-0.147502,4.669585,1.591619,0.254411,1,0
16873,10776,1.184832,0.558659,-0.409070,0.164328,-0.440434,0.455011,0.225590,0.455728,0.263855,0.313955,-0.147502,-0.218673,-0.078031,0.254411,1,1
16874,3612,1.032523,0.387063,4.043523,0.421658,1.128996,0.455011,0.305159,0.455728,0.129972,0.313955,0.890157,-0.218673,-0.078031,0.254411,1,3


In [11]:
from sklearn.metrics import classification_report

In [12]:
feat_cols = [
    'Age', 'Workclass', 'Fnlwgt', 'Education', 'Education-num', 'Marital-status',
    'Occupation', 'Relationship', 'Race', 'Sex', 'Capital-gain', 'Capital-loss',
    'Hours-per-week', 'Native-country'
]

BASE_RESULTS_PATH = Path("adult_analysis_results_lhs")
BASE_RESULTS_PATH.mkdir(exist_ok=False)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    # for each (target, cluster)...
    for ind, (targ, clus) in enumerate(adult_test_subset_cluster[['Target', 'Cluster']].values):
        ind_res_path = BASE_RESULTS_PATH / f"{ind:02d}_t{targ}_c{clus}"
        ind_res_path.mkdir(exist_ok=True)
        print(f"{ind=} {targ=} {clus=} {ind_res_path}")
        # retrieve the attack models
        attack_0, attack_1 = get_attack_mod(ind)

        # TRAIN
        train_path = ind_res_path / "train"
        train_path.mkdir(exist_ok="true")
        # get the subset of the training set
        train_subset = get_subset_target_cluster(adult_train, targ, clus)

        probs_train = black_box.predict_proba(train_subset[feat_cols].to_numpy())
        ones_train, zeroes_train = split_probs_array(probs_train)
        assert len(zeroes_train) + len(ones_train) == len(probs_train)

        if len(zeroes_train) > 0:
            atk_0_train = attack_0.predict(zeroes_train)
            report_0_train = classification_report(["in"]*len(atk_0_train), atk_0_train)
            with open(str(train_path / "0.txt"), "w") as f:
                f.write(report_0_train)

        if len(ones_train) > 0:
            atk_1_train = attack_1.predict(ones_train)
            report_1_train = classification_report(["in"]*len(atk_1_train), atk_1_train)
            with open(str(train_path / "1.txt"), "w") as f:
                f.write(report_1_train)
        
        l0tr = len(zeroes_train)
        l1tr = len(ones_train)
        
        train_results = None
        if l0tr == 0:
            train_results = atk_1_train
        elif l1tr == 0:
            train_results = atk_0_train
        else:
            train_results = np.concatenate((atk_0_train, atk_1_train))
        with open(str(train_path / "01.txt"), "w") as f:
            report_01_train = classification_report(["in"]*len(train_results), train_results)
            f.write(report_01_train)

        

        #TEST
        test_path = ind_res_path / "test"
        test_path.mkdir(exist_ok="true")
        # get test subset of the test set
        test_subset  = get_subset_target_cluster(adult_test, targ, clus)

        probs_test = black_box.predict_proba(test_subset[feat_cols].to_numpy())
        ones_test, zeroes_test = split_probs_array(probs_test)
        assert len(zeroes_test) + len(ones_test) == len(probs_test)

        if len(zeroes_test) > 0:
            atk_0_test = attack_0.predict(zeroes_test)
            report_0_test = classification_report(["out"] * len(atk_0_test), atk_0_test)
            with open(str(test_path / "0.txt"), "w") as f:
                f.write(report_0_test)

        if len(ones_test) > 0:
            atk_1_test = attack_0.predict(ones_test)
            report_1_test = classification_report(["out"] * len(atk_1_test), atk_1_test)
            with open(str(test_path / "1.txt"), "w") as f:
                f.write(report_1_test)
        
        l0te = len(zeroes_test)
        l1te = len(ones_test)
        test_results = None
        if l0te == 0:
            test_results = atk_1_test
        elif l1te == 0:
            test_results = atk_0_test
        else:
            test_results = np.concatenate((atk_0_test, atk_1_test))
            
        with open(str(test_path / "01.txt"), "w") as f:
            report_01_train = classification_report(["out"]*len(test_results), test_results)
            f.write(report_01_train)
        
        # Concatenating test and train
        with open(str(ind_res_path / "traintest_full.txt"), "w") as f:
            full = np.concatenate((train_results, test_results))
            mask = ["in"] * len(train_results) + ["out"] * len(test_results)
            f.write(classification_report(mask, full))

ind=0 targ=0 clus=0 adult_analysis_results_lhs/00_t0_c0
ind=1 targ=0 clus=0 adult_analysis_results_lhs/01_t0_c0
ind=2 targ=0 clus=0 adult_analysis_results_lhs/02_t0_c0
ind=3 targ=0 clus=0 adult_analysis_results_lhs/03_t0_c0
ind=4 targ=0 clus=0 adult_analysis_results_lhs/04_t0_c0
ind=5 targ=0 clus=1 adult_analysis_results_lhs/05_t0_c1
ind=6 targ=0 clus=1 adult_analysis_results_lhs/06_t0_c1
ind=7 targ=0 clus=1 adult_analysis_results_lhs/07_t0_c1
ind=8 targ=0 clus=1 adult_analysis_results_lhs/08_t0_c1
ind=9 targ=0 clus=1 adult_analysis_results_lhs/09_t0_c1
ind=10 targ=0 clus=2 adult_analysis_results_lhs/10_t0_c2
ind=11 targ=0 clus=2 adult_analysis_results_lhs/11_t0_c2
ind=12 targ=0 clus=2 adult_analysis_results_lhs/12_t0_c2
ind=13 targ=0 clus=2 adult_analysis_results_lhs/13_t0_c2
ind=14 targ=0 clus=2 adult_analysis_results_lhs/14_t0_c2
ind=15 targ=0 clus=3 adult_analysis_results_lhs/15_t0_c3
ind=16 targ=0 clus=3 adult_analysis_results_lhs/16_t0_c3
ind=17 targ=0 clus=3 adult_analysis_resul

# Using a Voting classifier

In [19]:
from sklearn.ensemble import VotingClassifier
from sklearn.utils.validation import check_is_fitted

In [13]:
BASE_VOTING_PATH = Path("voting_lhs")
BASE_VOTING_PATH.mkdir(exist_ok=False)

def hard_classify(predictionslist):
    def count(x):
        ins = [h for h in x if h == "in"]
        if len(ins) > 2:
            return "in"
        else:
            return "out"
    arr = np.asarray(predictionslist)
    return np.apply_along_axis(count, axis=0, arr=arr)

def soft_classify(predictionslist):
    def sum_probs(x):
        pass
    arr = np.asarray(predictionslist)
    return np.apply_along_axis(sum_probs, axis=0, arr=arr)

for a in adult_test_subset_cluster[['Target', 'Cluster']].groupby(['Target', 'Cluster']):
    indices = a[1].index.values.tolist()
    t,c = a[0][0], a[0][1]
    print(f"{t=} {c=} {indices=}")
    atk0 = []
    atk1 = []
    for ind in indices:
        a0, a1 = get_attack_mod(ind)
        atk0.append(a0)
        atk1.append(a1)
    
    
    train_subset = get_subset_target_cluster(adult_train, t, c)
    test_subset  = get_subset_target_cluster(adult_test, t, c)
    
    probs_train = black_box.predict_proba(train_subset[feat_cols].to_numpy())
    probs_test = black_box.predict_proba(test_subset[feat_cols].to_numpy())
    
    ones_train, zeroes_train = split_probs_array(probs_train)
    ones_test, zeroes_test = split_probs_array(probs_test)
    
    
    atk_0_train, atk_1_train, atk_0_test, atk_1_test = [], [], [], []
    if len(zeroes_train) > 0:
        atk_0_train = hard_classify([model.predict(zeroes_train) for model in atk0])

    if len(ones_train) > 0:
        atk_1_train = hard_classify([model.predict(ones_train) for model in atk1])
    
    if len(zeroes_test) > 0:
        atk_0_test = hard_classify([model.predict(zeroes_test) for model in atk0])

    if len(ones_test) > 0:
        atk_1_test = hard_classify([model.predict(ones_test) for model in atk1])
    
    print(f"{len(atk_0_train)=} {len(atk_1_train)=}")
    train_results = np.concatenate((atk_0_train, atk_1_train))
    train_labels  = ["in"] * len(train_results)
    test_results  = np.concatenate((atk_0_test, atk_1_test))
    test_labels   = ["out"] * len(test_results)
    
    full_results  = np.concatenate((train_results, test_results))
    full_labels   = np.concatenate((train_labels, test_labels))
    
    report_full = classification_report(full_labels, full_results)
    
    with open(str(BASE_VOTING_PATH / f"t{t}_c{c}.txt"), "w") as f:
        f.write(report_full)

t=0 c=0 indices=[0, 1, 2, 3, 4]


/home/gerardozinno/.pyenv/versions/3.9.9/envs/ml-environment/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/gerardozinno/.pyenv/versions/3.9.9/envs/ml-environment/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.0.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


len(atk_0_train)=2 len(atk_1_train)=4126
t=0 c=1 indices=[5, 6, 7, 8, 9]


/home/gerardozinno/.pyenv/versions/3.9.9/envs/ml-environment/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/gerardozinno/.pyenv/versions/3.9.9/envs/ml-environment/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/gerardozinno/.pyenv/versions/3.9.9/envs/ml-environment/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` paramete

len(atk_0_train)=421 len(atk_1_train)=2949
t=0 c=2 indices=[10, 11, 12, 13, 14]


...

len(atk_0_train)=721 len(atk_1_train)=1062
t=1 c=2 indices=[35, 36, 37, 38, 39]


/home/gerardozinno/.pyenv/versions/3.9.9/envs/ml-environment/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/gerardozinno/.pyenv/versions/3.9.9/envs/ml-environment/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/gerardozinno/.pyenv/versions/3.9.9/envs/ml-environment/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` paramete

len(atk_0_train)=83 len(atk_1_train)=0
t=1 c=3 indices=[40, 41, 42, 43, 44]


/home/gerardozinno/.pyenv/versions/3.9.9/envs/ml-environment/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/gerardozinno/.pyenv/versions/3.9.9/envs/ml-environment/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/gerardozinno/.pyenv/versions/3.9.9/envs/ml-environment/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` paramete

len(atk_0_train)=1602 len(atk_1_train)=323


In [26]:
a0.classes_

'in'

In [28]:
atk0

[('zero_0',
  RandomForestClassifier(criterion='entropy', max_depth=100, min_samples_leaf=10,
                         min_samples_split=5)),
 ('zero_1',
  RandomForestClassifier(max_depth=100, min_samples_leaf=20, min_samples_split=5)),
 ('zero_2',
  RandomForestClassifier(criterion='entropy', max_depth=100, min_samples_leaf=10,
                         min_samples_split=5)),
 ('zero_3',
  RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=500,
                         max_features='sqrt', min_samples_leaf=50,
                         min_samples_split=50, n_estimators=500)),
 ('zero_4',
  RandomForestClassifier(criterion='entropy', max_depth=500, min_samples_leaf=10,
                         min_samples_split=50, n_estimators=350))]

In [29]:
aaa = [x[1] for x in atk0]

In [30]:
aaa

[RandomForestClassifier(criterion='entropy', max_depth=100, min_samples_leaf=10,
                        min_samples_split=5),
 RandomForestClassifier(max_depth=100, min_samples_leaf=20, min_samples_split=5),
 RandomForestClassifier(criterion='entropy', max_depth=100, min_samples_leaf=10,
                        min_samples_split=5),
 RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=500,
                        max_features='sqrt', min_samples_leaf=50,
                        min_samples_split=50, n_estimators=500),
 RandomForestClassifier(criterion='entropy', max_depth=500, min_samples_leaf=10,
                        min_samples_split=50, n_estimators=350)]

In [34]:
x_test

array([[-1.40441914,  0.26896013, -0.106742  , ..., -0.21867335,
        -0.07803133,  0.25441118],
       [ 0.80406006,  0.2946299 ,  0.80713136, ..., -0.21867335,
         1.59161922,  0.25441118],
       [-0.18594786,  0.21891843, -0.29131116, ..., -0.21867335,
        -0.24499639,  0.25441118],
       ...,
       [ 0.65175115,  0.21891843,  1.70669424, ..., -0.21867335,
         0.75679395,  0.25441118],
       [ 0.80406006,  0.21891843,  3.35167007, ..., -0.21867335,
         0.08893372,  0.25441118],
       [ 0.11866996,  0.21891843, -0.64206923, ..., -0.21867335,
        -0.07803133,  0.38888888]])

In [35]:
probs_train = black_box.predict_proba(train_subset[feat_cols].to_numpy())

In [36]:
probs_train

array([[9.90213854e-01, 9.78614599e-03],
       [9.70459682e-01, 2.95403176e-02],
       [9.99913420e-01, 8.65800866e-05],
       ...,
       [8.47961415e-01, 1.52038585e-01],
       [9.86203174e-01, 1.37968257e-02],
       [9.98158455e-01, 1.84154489e-03]])

In [37]:
ones_train, zeroes_train = split_probs_array(probs_train)

In [38]:
ones_train

array([[9.90213854e-01, 9.78614599e-03],
       [9.70459682e-01, 2.95403176e-02],
       [9.99913420e-01, 8.65800866e-05],
       ...,
       [8.47961415e-01, 1.52038585e-01],
       [9.86203174e-01, 1.37968257e-02],
       [9.98158455e-01, 1.84154489e-03]])

In [40]:
preds = [m.predict(ones_train) for m in aaa]

In [41]:
preds

[array(['in', 'in', 'in', ..., 'in', 'in', 'in'], dtype=object),
 array(['in', 'in', 'in', ..., 'in', 'in', 'in'], dtype=object),
 array(['in', 'in', 'in', ..., 'in', 'in', 'in'], dtype=object),
 array(['in', 'in', 'in', ..., 'in', 'in', 'in'], dtype=object),
 array(['in', 'in', 'in', ..., 'in', 'in', 'in'], dtype=object)]

In [43]:
asa = np.asarray(preds)

In [51]:
asa

array([['in', 'in', 'in', ..., 'in', 'in', 'in'],
       ['in', 'in', 'in', ..., 'in', 'in', 'in'],
       ['in', 'in', 'in', ..., 'in', 'in', 'in'],
       ['in', 'in', 'in', ..., 'in', 'in', 'in'],
       ['in', 'in', 'in', ..., 'in', 'in', 'in']], dtype=object)

In [54]:
np.apply_along_axis(lambda x: len(x), axis=0, arr=asa)

array([5, 5, 5, ..., 5, 5, 5])

In [57]:
def count(x):
        ins = [h for h in x if h == "in"]
        if len(ins) > 2:
            return "in"
        else:
            return "out"

In [59]:
count(asa[:,0])

'in'